# Exploring Ebay Car Sales Data

In this guided project, I will use this [dataset](https://www.kaggle.com/piumiu/used-cars-database-50000-data-points) which is provided by [dataquest](https://www.dataquest.io). It is a *smaller* and *dirtier* version of [dataset](https://www.kaggle.com/orgesleka/used-cars-database/data) from [eBay Kleinanzeigen](https://www.ebay-kleinanzeigen.de/).

My goals are to clean and analyze the included used car listing.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('../input/used-cars-database-50000-data-points/autos.csv', encoding = 'Latin-1')

autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


My observations about the dataset:

* There are 50,000 rows, 20 columns.
* Some columns have NULL values, but less than 20% of total records.
* Column names are [camelcase](https://en.wikipedia.org/wiki/Camel_case) instead of [snakecase](https://en.wikipedia.org/wiki/Snake_case#Examples_of_languages_that_use_snake_case_as_convention). They will be changed to camel case.
* Some columns contain numeric values stored as text such as: `price` and `odometer`. These values also will be converted to numeric.

## I. Cleaning Column Names:

To rename column names, we can use pandas [rename()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) method.

In [2]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [3]:
autos.rename(columns = {'yearOfRegistration':'registration_year', 'monthOfRegistration':'registration_month', 
                       'notRepairedDamage':'unrepaired_damage', 'dateCreated':'ad_created'}, inplace = True)

To change from camel case to snake case we can use function [underscore()](https://inflection.readthedocs.io/en/latest/_modules/inflection.html#underscore) from [Inflection](https://inflection.readthedocs.io/en/latest/index.html). It is a string transformation library. We will need import [re](https://docs.python.org/2/library/re.html) module before using the function.

In [4]:
import re

def underscore(word):
    """
    Make an underscored, lowercase form from the expression in the string.

    Example::

        >>> underscore("DeviceType")
        "device_type"

    As a rule of thumb you can think of :func:`underscore` as the inverse of
    :func:`camelize`, though there are cases where that does not hold::

        >>> camelize(underscore("IOError"))
        "IoError"

    """
    word = re.sub(r"([A-Z]+)([A-Z][a-z])", r'\1_\2', word)
    word = re.sub(r"([a-z\d])([A-Z])", r'\1_\2', word)
    word = word.replace("-", "_")
    return word.lower()

for item in autos.columns:
    col_name = item
    col_name = underscore(col_name)
    autos.rename(columns = {item : col_name}, inplace = True)
    
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

The reason why we should use snake case in Python, because [PEP8 Naming Conventions](https://www.python.org/dev/peps/pep-0008/#naming-conventions) suggests us to use lower case with underscore for function, method, variable and constant. Using naming convention is not only help to increase readability, but also let other developers understand your code easier.

## II. Initial Exploration and Cleaning:

In this part, I will determine which columns should be ignored, columns have numeric values but stored as text.

In [5]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


* Column `seller`, `offer_type`, `abtest`, `unrepaired_damage`, `nr_of_pictures`, `postal_code` contain text values and almost all values are the same. These columns are candidates to be ignored because they do not have useful information for analysis.

* Column `registration_year` has very weird values (e.g. `9999` and `1000`). I will take a closer look at this later.

* `price` and `odometer` have numeric data stored as text that needs to be converted.

In [6]:
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
autos['price'] = autos['price'].astype(float)
autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos['odometer'] = autos['odometer'].astype(int)
autos.rename(columns = {'odometer' : 'odometer_km'}, inplace = True)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990.0,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350.0,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350.0,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## III. Exploring Columns:

After converting `odometer_km` and `price`, we have to detect outliers (values that unrealistically high or low). There are many ways to archive our goals. You can read this [article](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba) for more details. Now, I am going to use **Z-Score** method to discover outliers.

You can also watch these videos [Mode, Median, Mean, Range, and Standard Deviation](https://www.youtube.com/watch?v=mk8tOD0t8M0), [Z-Scores and Percentiles](https://www.youtube.com/watch?v=uAxyI_XfqXk&t=4s) for more visualized explanations.

Simply speaking, **Z-score** tell us how far a certain value from the mean.

### Part 1. Exploring Price Column:

In [7]:
from scipy import stats

zp = np.abs(stats.zscore(autos['price']))
print(zp)

[0.01006038 0.00278538 0.00176688 ... 0.00698391 0.02714606 0.01785502]


Above numbers have not told us much. In most cases, if z-score of a value is less than -3 or greater than 3, that value will be identified as an outlier. We can use numpy [where()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html) to extract index of values that have z-score greater than 3.

In [8]:
for item in np.where(zp > 3):
    print(autos['price'].iloc[item])

2897     11111111.0
11137    10000000.0
24384    11111111.0
27371    12345678.0
39377    12345678.0
39705    99999999.0
42221    27322222.0
47598    12345678.0
47634     3890000.0
Name: price, dtype: float64


In [9]:
autos['price'].min()

0.0

Minium value in `price` column is `$0.0`. eBay is an auction site, the opening bid for an item could be `$1.0`. So we will keep all values between `$1.0` and `$3,889,999.0` by using [between()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html) method:

In [10]:
autos = autos[autos['price'].between(0, 3890000, inclusive = False)]
autos.describe()

,price,registration_year,power_ps,odometer_km,registration_month,nr_of_pictures,postal_code
count,4.857000e+04,48570.000000,48570.000000,48570.000000,48570.000000,48570.0,48570.000000
mean,6.002279e+03,2004.754231,117.194812,125770.022648,5.782294,0.0,50975.462219
std,1.444526e+04,88.639460,200.640396,39788.530021,3.685734,0.0,25747.686720
min,1.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.200000e+03,1999.000000,71.000000,125000.000000,3.000000,0.0,30657.000000
50%,3.000000e+03,2004.000000,107.000000,150000.000000,6.000000,0.0,49716.000000
75%,7.490000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71665.000000
max,1.300000e+06,9999.000000,17700.000000,150000.000000,12.000000,0.0,99998.000000


In [11]:
autos.shape

(48570, 20)

### Part 2. Exploring Odometer_km Column:

Just need to repeat above steps

In [12]:
zo = np.abs(stats.zscore(autos['odometer_km']))
print(zo)

[0.60897517 0.60897517 1.40167523 ... 3.03532867 2.15566912 0.60897517]


In [13]:
for item in np.where(zo > 3):
    print(autos['odometer_km'].iloc[item])

52       5000
76       5000
102      5000
106      5000
121      5000
         ... 
49722    5000
49844    5000
49845    5000
49865    5000
49997    5000
Name: odometer_km, Length: 836, dtype: int64


836 records are not too many (around 1.72% of total amount). I will exclude them from my dataset.

In [14]:
autos = autos[autos['odometer_km'].between(5000, 150001, inclusive = False)]
autos.shape

(47734, 20)

### Part 3. Exploring the Date Columns:

We have 5 columns that represent date values:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

`registration_month` and `registration_year` are represented as numeric values. The other three columns are represented as timestamp.

In [15]:
autos[['date_crawled', 'ad_created', 'last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


The first 10 characters represent the day (e.g. `2016-03-26`). We can extract the day by using `Series.str[:10]`, chains to [value_counts(normalize = True)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) to generate a distribution, and then sort by the index with [sort_index()](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.Series.sort_index.html). [Normalization](https://medium.com/@urvashilluniya/why-data-normalization-is-necessary-for-machine-learning-models-681b65a05029) will help us to change values in the three columns to a common scale, therefore we can compare them. We also need to count Null values in these columns with [isnull()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) and [sum()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html) to specify should we exclude missing values or not. [If the number of the cases is less than 5% of the sample](https://www.statisticssolutions.com/missing-values-in-data/), we can drop them by set `dropna = True` in `value_count()`.

In [16]:
autos[['date_crawled', 'ad_created', 'last_seen']].isnull().sum()

date_crawled    0
ad_created      0
last_seen       0
dtype: int64

In [17]:
autos['date_crawled'].str[:10].value_counts(normalize = True).sort_index()

2016-03-05    0.025349
2016-03-06    0.014099
2016-03-07    0.036033
2016-03-08    0.033414
2016-03-09    0.033142
2016-03-10    0.032178
2016-03-11    0.032597
2016-03-12    0.037122
2016-03-13    0.015628
2016-03-14    0.036515
2016-03-15    0.034294
2016-03-16    0.029350
2016-03-17    0.031466
2016-03-18    0.012842
2016-03-19    0.034755
2016-03-20    0.037939
2016-03-21    0.037499
2016-03-22    0.032786
2016-03-23    0.032325
2016-03-24    0.029224
2016-03-25    0.031676
2016-03-26    0.032220
2016-03-27    0.031068
2016-03-28    0.034671
2016-03-29    0.034106
2016-03-30    0.033624
2016-03-31    0.031717
2016-04-01    0.033896
2016-04-02    0.035530
2016-04-03    0.038652
2016-04-04    0.036557
2016-04-05    0.013156
2016-04-06    0.003163
2016-04-07    0.001404
Name: date_crawled, dtype: float64

Data was crawled daily from beginning of March 2016 to beginning of April 2016. The distribution between each days is almost identical.

In [18]:
autos['ad_created'].str[:10].value_counts(normalize = True).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038903
2016-04-04    0.036934
2016-04-05    0.011857
2016-04-06    0.003247
2016-04-07    0.001257
Name: ad_created, Length: 76, dtype: float64

Before March 2016, not many ads were created on eBay. From March 2016 to April 2016, ads were uploaded everyday. The frequency of days is also nearly equal. We can see that percentage of crawling and creating data are similar.

In [19]:
autos['last_seen'].str[:10].value_counts(normalize = True).sort_index()

2016-03-05    0.001027
2016-03-06    0.004357
2016-03-07    0.005405
2016-03-08    0.007437
2016-03-09    0.009595
2016-03-10    0.010684
2016-03-11    0.012423
2016-03-12    0.023463
2016-03-13    0.008883
2016-03-14    0.012716
2016-03-15    0.015775
2016-03-16    0.016362
2016-03-17    0.028177
2016-03-18    0.007395
2016-03-19    0.015817
2016-03-20    0.020803
2016-03-21    0.020677
2016-03-22    0.021431
2016-03-23    0.018519
2016-03-24    0.019797
2016-03-25    0.019211
2016-03-26    0.016739
2016-03-27    0.015586
2016-03-28    0.020656
2016-03-29    0.022353
2016-03-30    0.024825
2016-03-31    0.023778
2016-04-01    0.022919
2016-04-02    0.024909
2016-04-03    0.025286
2016-04-04    0.024364
2016-04-05    0.124649
2016-04-06    0.221896
2016-04-07    0.132086
Name: last_seen, dtype: float64

`last_seen` tells us the day ads were removed from eBay. It could be the car was sold. Most of the ads were end at April 2016 (around 50%). It's not likely there was a spike on sales, but more likely because of crawling period ending. Now, we continue with `registation_year`.

In [20]:
autos['registration_year'].describe()

count    47734.000000
mean      2004.469770
std         82.330936
min       1910.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

I once mentioned that `registration_year` has some weired values (e.g. `9999`). We can use z-score to remove those outliers.

In [21]:
zr = np.abs(stats.zscore(autos['registration_year']))
for item in np.where(zr > 3):
    print(autos['registration_year'].iloc[item])

8012     9999
8360     6200
25003    8888
27618    5911
33950    9999
38076    9999
49910    9000
Name: registration_year, dtype: int64


Any vehicles with registration year above 2016 are incorrect. We cannot have first registration greater than `last_seen`.

In [22]:
autos = autos[autos['registration_year'].between(1910, 2016)]
autos.shape

(45901, 20)

In [23]:
autos['registration_year'].value_counts(normalize = True).sort_index()

1910    0.000044
1931    0.000022
1934    0.000022
1937    0.000044
1941    0.000044
          ...   
2012    0.028409
2013    0.017342
2014    0.013943
2015    0.006688
2016    0.024923
Name: registration_year, Length: 74, dtype: float64

From 1994 onwards, There was an increase of car registration. Especially in early years of the 21st century and then decreased gradually until 2016.

## III. Exploring Price by Brand:

Next thing is exploring average of price by Brand. First of all, we will chose top 5 most popular Brands.

In [24]:
brand_count = autos['brand'].value_counts(normalize = True)
common_brand = brand_count[brand_count > 0.05].index
print(common_brand)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [25]:
autos['brand'].value_counts(normalize = True)

volkswagen        0.211564
bmw               0.110259
opel              0.107231
mercedes_benz     0.096882
audi              0.087297
ford              0.069933
renault           0.047080
peugeot           0.030086
fiat              0.025490
seat              0.018278
skoda             0.016383
nissan            0.015250
mazda             0.015207
smart             0.014292
citroen           0.014074
toyota            0.012788
hyundai           0.010087
volvo             0.009215
sonstige_autos    0.008954
mini              0.008714
mitsubishi        0.008170
honda             0.007799
kia               0.007146
alfa_romeo        0.006579
suzuki            0.005926
porsche           0.005904
chevrolet         0.005534
chrysler          0.003442
dacia             0.002636
daihatsu          0.002440
jeep              0.002288
subaru            0.002157
land_rover        0.002070
saab              0.001678
jaguar            0.001569
daewoo            0.001481
rover             0.001351
t

By far, Volkswagen is the most popluar brand which has number of cars several times higher than other manufacturers. German brand takes 4 of 5 places in the top 5 list. Next is to calculate average price of each brand.

In [26]:
mean_by_brand = {}

for brand in common_brand:
    brand_df = autos.loc[autos['brand'] == brand]
    mean_val = brand_df['price'].mean()
    mean_by_brand[brand] = int(mean_val)
    
print(mean_by_brand)

{'volkswagen': 5624, 'bmw': 8548, 'opel': 2958, 'mercedes_benz': 8602, 'audi': 9280, 'ford': 3989}


Audi, Mercedes-Benz, and BMW are expensive. Ford and Opel are cheaper. Volkswagen is between, maybe because its popularity.

## IV. Exploring the Link between Price and Odometer:

To understand the link between average mileage and mean price we need compare two series objects. We can combine data from both series into a dataframe by using pandas [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) and [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [27]:
mileage_by_brand = {}

for brand in common_brand:
    brand_df = autos.loc[autos['brand'] == brand]
    avg_val = brand_df['odometer_km'].mean()
    mileage_by_brand[brand] = int(avg_val)

print(mileage_by_brand)

{'volkswagen': 130660, 'bmw': 134516, 'opel': 131835, 'mercedes_benz': 132372, 'audi': 130210, 'ford': 126272}


In [28]:
mean_series = pd.Series(mean_by_brand).sort_values(ascending = False)
print(mean_series)

audi             9280
mercedes_benz    8602
bmw              8548
volkswagen       5624
ford             3989
opel             2958
dtype: int64


In [29]:
mileage_series = pd.Series(mileage_by_brand).sort_values(ascending = False)
print(mileage_series)

bmw              134516
mercedes_benz    132372
opel             131835
volkswagen       130660
audi             130210
ford             126272
dtype: int64


In [30]:
mmb = pd.DataFrame(mean_series, columns = ['mean_price'])
mmb['average_mileage'] = mileage_series
mmb

,mean_price,average_mileage
audi,9280,130210
mercedes_benz,8602,132372
bmw,8548,134516
volkswagen,5624,130660
ford,3989,126272
opel,2958,131835


Average mileage between brands are not too different. Instead, with high-class brand such as Audi, Mercedes-Benz and BMW, the average mileage are higher than the others. It seems that people who own luxury car, tend to use their vehicle longer. 

*The purpose of this project is mainly to practice what I have learned from [dataquest.io](dataquest.io) - Python for Data Science: Pandas and Numpy Fundamentals. Many techniques, contents in this project were guided by dataquest.io and the following [solution](https://github.com/dataquestio/solutions/blob/master/Mission294Solutions.ipynb).*